In [45]:
def edge_potential(A, b, L):

    # chapter 2.2.7.5 in Murphy, Probabilistic Machine Learning: Advanced Topics
    # potential over (x_{t - 1}, x_t) or (x_t, y_t), in that order

    g = - 0.5 * (b @ L @ b + b.size * np.log(2 * math.pi) + np.log(np.linalg.det(np.linalg.inv(L)))) # is b.size correct dim?
    K = np.block([[A.T @ L @ A, - A.T @ L], [- L @ A, L]]) # is A transpose the right way?
    h = np.concatenate((- A.T @ L @ b, L @ b))

    return g, K, h

def condition_potential_on_evidence(g, K, h, y):

    # chapter 2.2.7.4 in Murphy, Probabilistic Machine Learning: Advanced Topics

    g_new = g + h[-y.size:] @ y - 0.5 * y @ K[-y.size:, -y.size:] @ y
    K_new = K[:-y.size, :-y.size]
    h_new = h[:-y.size] - K[:-y.size, -y.size:] @ y

    return g_new, K_new, h_new


def log_normaliser(J, h, diagonal_boost = 1e-9):

    # https://en.wikipedia.org/wiki/Exponential_family

    # https://math.stackexchange.com/questions/3158303/using-cholesky-decomposition-to-compute-covariance-matrix-determinant
    L = np.linalg.cholesky(J + diagonal_boost * np.eye(J.shape[-1]))
    half_log_det_precision = np.log(np.diagonal(L, axis1 = -2, axis2 = -1)).sum(-1)

    return 0.5 * h @ np.linalg.solve(J, h) - half_log_det_precision

import numpy as np
import math
import tensorflow_probability.substrates.jax.distributions as tfd

C = np.random.randn(3,3)
d = np.random.randn(3,)
R = np.random.randn(3,3)
R = R @ R.T * 10
y = np.random.randn(3,)
x = np.random.randn(3,)

gE, KE, hE = edge_potential(C, d, np.linalg.inv(R))
gC, KC, hC = condition_potential_on_evidence(gE, KE, hE, y)
Sig = np.linalg.inv(KC)
mu = Sig @ hC

# this is a probability distribution wrt y, so this computation is valid
noise = tfd.MultivariateNormalFullCovariance(loc=d, covariance_matrix=R)
obs_ll_from_likelihood = noise.log_prob(y - C @ x)

# obs_ll_from_likelihood can also be computed as follows:
# here we evaluate the potential directly (Murph 2, equation 2.112)
obs_ll_from_likelihood_version2 = gC + x @ hC - 0.5 * x @ KC @ x

# this is not a probability distribution wrt x, it is an unormalised potential on x, 
# so this is not a valid computation as MVN assumes that this is a probability distribution wrt x
noise = tfd.MultivariateNormalFullCovariance(loc=mu, covariance_matrix=Sig)
obs_ll_from_potential = noise.log_prob(x)

# we can compute obs_ll_from_likelihood from obs_ll_from_potential as follows:
# here we subtract the incorrectly assumed scaling factor (the inverse of the normalising constant for the potential) 
# and then add the true scaling factor gC
assumed_scaling = (np.log((2 * math.pi) **(-3/2)) - log_normaliser(KC, hC))
corrected_obs_ll_from_potential = obs_ll_from_potential - assumed_scaling + gC

# NB log_normaliser(KC, hC) = -(np.log(np.linalg.det(Sig)**(-1/2)) -0.5 * mu @ KC @ mu)
# log_normaliser for a Gaussian is not the full normalising constant
# it does include not base measure (the 2 * pi term), which we add in to get assumed_scaling

# because obs_ll_from_potential is not equal to obs_ll_from_likelihood,
# the true marginal likelihood is not equal to the one which assumes the potentials on x are pdfs on y

In [44]:
print(obs_ll_from_likelihood)
print(obs_ll_from_likelihood_version2)
print(corrected_obs_ll_from_potential)
print(obs_ll_from_potential)

-5.9088416
-5.908841493889845
-5.908842
-8.008678


In [ ]:
# posterior.log_normalizer in svae is not the same as the marginal likelihood, lps, computed
# using the actual likelihood (for the reasons above), but the correction can be computed:

lps - (posterior.log_normalizer + ll_correction.sum())

def edge_potential(self, A, b, L):

    # chapter 2.2.7.5 in Murphy, Probabilistic Machine Learning: Advanced Topics
    # potential over (x_{t - 1}, x_t) or (x_t, y_t), in that order

    g = - 0.5 * (b @ L @ b + b.size * np.log(2 * math.pi) + np.log(np.linalg.det(np.linalg.inv(L)))) # is b.size correct dim?
    K = np.block([[A.T @ L @ A, - A.T @ L], [- L @ A, L]]) # is A transpose the right way?
    h = np.concatenate((- A.T @ L @ b, L @ b))

    return g, K, h

def condition_potential_on_evidence(self, g, K, h, y):

    # chapter 2.2.7.4 in Murphy, Probabilistic Machine Learning: Advanced Topics

    g_new = g + h[-y.size:] @ y - 0.5 * y @ K[-y.size:, -y.size:] @ y
    K_new = K[:-y.size, :-y.size]
    h_new = h[:-y.size] - K[:-y.size, -y.size:] @ y

    return g_new, K_new, h_new

def log_normaliser(self, J, h, diagonal_boost = 1e-9):

    # https://en.wikipedia.org/wiki/Exponential_family

    # https://math.stackexchange.com/questions/3158303/using-cholesky-decomposition-to-compute-covariance-matrix-determinant
    L = np.linalg.cholesky(J + diagonal_boost * np.eye(J.shape[-1]))
    half_log_det_precision = np.log(np.diagonal(L, axis1 = -2, axis2 = -1)).sum(-1)

    return 0.5 * h @ np.linalg.solve(J, h) - half_log_det_precision

def correct_ll(self, C, d, R, y):

    gE, KE, hE = self.edge_potential(C, d, np.linalg.inv(R))
    gC, KC, hC = self.condition_potential_on_evidence(gE, KE, hE, y)
    ll_correction = gC - (np.log((2 * np.pi) **(-3/2)) - self.log_normaliser(KC, hC))

    return ll_correction

ll_correction = vmap(self.correct_ll, in_axes=(None,None,None,0))(params["C"], params["d"], params["R"], data)